In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import api_key as api

api_key = api.deepseek_api_key()
base_url= 'https://api.deepseek.com/v1'
model_name='deepseek-chat'

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    ("user", "{input}")
])

llm = ChatOpenAI(
    base_url=base_url,
    api_key=api_key,
    model_name=model_name,
    streaming=True
)

output_parser = StrOutputParser()

chain = prompt | llm | output_parser
response = chain.stream({"input": "世界轴心时代的四位圣人"})

for value in response:
    print(value, end='', flush=True)

世界轴心时代的四位圣人通常指的是在公元前8世纪至前3世纪这个被历史学家称为“轴心时代”（Axial Age）的时期，在不同文明中出现的四位伟大的思想家和哲学家。这四位圣人分别是：

1. 孔子（Confucius）：中国春秋时期的思想家、教育家，儒家学派的创始人。他的思想强调仁爱、礼义、中庸之道，对中国乃至东亚文化产生了深远的影响。

2. 佛陀（Buddha）：即释迦牟尼，古印度的宗教导师，佛教的创始人。他提出了四圣谛和八正道，强调通过修行达到涅槃，解脱生死轮回。

3. 苏格拉底（Socrates）：古希腊哲学家，以其对话式的哲学探究方法和对道德、知识的追求而闻名。他的思想通过其弟子柏拉图的著作得以流传，对西方哲学产生了深远影响。

4. 耶稣基督（Jesus Christ）：基督教的中心人物，被认为是上帝的儿子，他的教导和生平事迹构成了基督教信仰的核心。耶稣的教导强调爱、宽恕和牺牲，对西方文化和世界历史产生了巨大影响。

这四位圣人的思想和教导在各自的文化中产生了深远的影响，并且随着时间的推移，他们的影响力扩展到了世界各地，成为人类文明发展的重要组成部分。

In [4]:
from langchain_community.vectorstores import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstore = Chroma.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import os
import api_key as api

os.environ["OPENAI_API_KEY"] = api.openai_api_key()

vectorstore = Chroma.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

api_key = api.deepseek_api_key()
base_url= 'https://api.deepseek.com/v1'
model_name='deepseek-chat'

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(
    base_url=base_url,
    api_key=api_key,
    model_name=model_name,
    streaming=True
)

output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

chain = setup_and_retrieval | prompt | llm | output_parser

chain.invoke("where did harrison work?")

# response = chain.stream({"question": "where did harrison work?"})
# for value in response:
#     print(value, end='', flush=True)

' Based on the provided context, Harrison worked at Kensho. This information is consistent across all the documents that mention Harrison\'s employment, with each document stating "harrison worked at kensho."'

In [7]:
chain.invoke("where did bears work?")

' Based on the provided context, there is no information about bears working at any specific location. The context only mentions that bears like to eat honey and that Harrison worked at Kensho. Therefore, it is not possible to answer the question about where bears worked.'

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import os
import api_key as api

os.environ["OPENAI_API_KEY"] = api.openai_api_key()

vectorstore = Chroma.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

api_key = api.deepseek_api_key()
base_url= 'https://api.deepseek.com/v1'
model_name='deepseek-chat'

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(
    base_url=base_url,
    api_key=api_key,
    model_name=model_name,
    streaming=True
)

output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

chain = setup_and_retrieval | prompt | model | output_parser

response = chain.stream("where did harrison work?")
for value in response:
    print(value, end='', flush=True)

 Based on the provided context, Harrison worked at Kensho. The information is consistent across all four documents, each stating that "Harrison worked at Kensho."<generator object RunnableSequence.stream at 0x146ea85f0>


In [16]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader("/Users/Daglas/Downloads/20230705舒伟杰培训HAZOP.md")
docs = loader.load()

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=500,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.split_documents(docs)
len(texts)

63

In [17]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings

loader = TextLoader("/Users/Daglas/Downloads/20230705舒伟杰培训HAZOP.md")
docs = loader.load()

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=500,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.split_documents(docs)

# save to disk
vectorstore = Chroma.from_documents(
    documents=texts,
    embedding=OpenAIEmbeddings(),
    persist_directory="./chroma_db"
)

In [19]:
# load from disk
vectorstore = Chroma(
        persist_directory="./chroma_db",
        embedding_function=OpenAIEmbeddings(),)

retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

query="什么是 HAZOP 分析"

results = retriever.get_relevant_documents(query)

len(results)
#4

for doc in results:
  print(doc)


page_content='作为 HAZOP 分析的一环，每个人都需要带着自己独立思考的角度去审视问题，并最终得出结论。当你们进行分析时，如果无法找到答案，那正是你们进行决策和设计改进的好机会，因为毕竟，我们的目标是将风险降到最低。' metadata={'source': '/Users/Daglas/Downloads/20230705舒伟杰培训HAZOP.md'}
page_content='在进行 HAZOP 分析时，最初的阶段是搜集所有必要的数据，包括流程图和物质特性等信息。这些数据将成为分析的基础。之后，通过一系列的讨论和审查会议，识别潜在的问题，并提出相应的建议。在这个过程中，细致的记录和详尽的分析是不可或缺的，以确保不会漏掉任何关键的信息。\n\n此外，针对异常情况的处理，我们需要仔细分析造成异常的原因，并考虑可能的后果。例如，如果某个阀门故障关闭了，我们必须预见到它可能会导致压力超标。同样，如果在进料过程中出现故障，持续加料可能会导致超压现象的发生。\n\n这里的核心概念是，我们进行的是一项安全评估，并不是工艺计算。安全评估的目的是确保操作中的安全，而非单纯地追求工艺的效率。当前，安全法准则可能与传统工艺计算有所差异，但我们的最终目标始终是确保操作的安全可控。在 HAZOP 分析中，我们不仅需要掌握技术知识和经验，还需要关注当前安全法规的要求，并据此做出合适的操作决策。' metadata={'source': '/Users/Daglas/Downloads/20230705舒伟杰培训HAZOP.md'}
page_content='首先，让我们理解一下什么是 HAZOP 分析。它是危险与可操作性研究（Threat and Operability Studies, THA）或者威胁与可操作性分析（Threat and Operability Analysis, THA），通常简称为 HAZOP 分析。\n\nHAZOP 由哪些关键要素组成呢？它是基于风险的一种安全分析方法，通常需要以团队合作的形式进行，利用集体智慧，所以需要组织会议，不是单独个人能够完成的。一个 HAZOP 团队往往包括分析主席、记录员、工艺设计人员、现场操作人员以及其他相关人士，确保风险分析能够全面。\n\nHAZOP 方法本身非常简单，团队成员共同考虑在哪些环节可能会遇到危险

In [28]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import os
import api_key as api

os.environ["OPENAI_API_KEY"] = api.openai_api_key()

# load from disk
vectorstore = Chroma(
        persist_directory="./chroma_db",
        embedding_function=OpenAIEmbeddings(),)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

api_key = api.deepseek_api_key()
base_url= 'https://api.deepseek.com/v1'
model_name='deepseek-chat'

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(
    base_url=base_url,
    api_key=api_key,
    model_name=model_name,
    streaming=True
)

output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

chain = setup_and_retrieval | prompt | model | output_parser

response = chain.stream("什么是 HAZOP 分析，结合实际案例详细讲解，一步一步来")
for value in response:
    print(value, end='', flush=True)

 HAZOP（危害与可操作性分析）是一种系统性的风险评估方法，用于识别和评估工艺流程中可能存在的危险和操作问题。它通过一系列的讨论和审查会议，结合专业知识和经验，对流程图和物质特性等数据进行分析，以识别潜在的问题并提出改进建议。HAZOP分析的目的是确保操作的安全性，降低风险，并防止事故的发生。

以下是一个简化的HAZOP分析案例步骤：

1. **数据搜集**：首先，搜集所有必要的数据，包括工艺流程图、设备规格、操作条件、物质特性等。例如，对于一个化工厂的某个生产单元，需要收集该单元的工艺流程图和涉及的化学物质的安全数据表。

2. **团队组建**：组建一个跨学科的团队，包括工艺工程师、安全专家、操作人员等，他们将共同参与分析过程。

3. **引导词应用**：使用一系列的引导词（如“没有”、“过多”、“相反”等）来系统地审查流程中的每个节点。例如，对于一个反应器，使用引导词“没有”来考虑如果反应物没有按时加入会发生什么。

4. **识别问题**：通过引导词的应用，团队识别出潜在的问题。例如，如果反应物没有按时加入，可能会导致反应器内温度升高，从而引发安全风险。

5. **分析后果**：对于识别出的每个问题，分析其可能的后果。例如，温度升高可能导致反应器压力升高，甚至引发爆炸。

6. **提出建议**：基于分析，提出改进措施或建议。例如，可以建议安装一个自动控制系统，确保反应物按时加入，或者增加一个安全阀来防止压力超标。

7. **记录和审查**：详细记录分析过程和结果，并进行审查以确保没有遗漏关键信息。

8. **实施和跟踪**：将提出的建议付诸实施，并跟踪其效果，确保风险得到有效控制。

在实际操作中，HAZOP分析是一个迭代过程，可能需要多次会议和讨论来完善分析结果。此外，分析过程中可能会使用“五个为什么”的方法来深入探究问题的根本原因，如上述文档中提到的迟到案例，通过连续追问“为什么”来找到问题的根源。这种方法有助于确保提出的解决方案能够真正解决问题，而不仅仅是应对表面现象。

In [38]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable, RunnableParallel, RunnablePassthrough
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

import os
import api_key as api

os.environ["OPENAI_API_KEY"] = api.openai_api_key()

# load from disk
vectorstore = Chroma(
        persist_directory="./chroma_db",
        embedding_function=OpenAIEmbeddings(),)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

api_key = api.deepseek_api_key()
base_url= 'https://api.deepseek.com/v1'
model_name='deepseek-chat'

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(
    base_url=base_url,
    api_key=api_key,
    model_name=model_name,
    streaming=True
)

output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

chain = setup_and_retrieval | prompt | model | output_parser

response = chain.stream("什么是 HAZOP 分析，结合实际案例详细讲解，一步一步来")
for value in response:
    print(value, end='', flush=True)

 HAZOP（危害与可操作性分析）是一种系统性的风险评估方法，用于识别和评估工艺流程中可能存在的危险和操作问题。它通过一系列的引导词（如“没有”、“过多”、“相反”等）与流程参数（如“压力”、“温度”、“流量”等）相结合，来激发团队成员的创造性思维，从而发现潜在的问题和风险。

以下是HAZOP分析的步骤，结合一个实际案例进行详细讲解：

### 步骤1：搜集数据
首先，需要搜集所有必要的数据，包括流程图、物质特性、操作条件、设备规格等。例如，假设我们要分析一个化工厂的氯气生产流程。我们需要收集该流程的详细图纸、氯气的物理化学特性、操作参数（如温度、压力）、设备清单等。

### 步骤2：组建团队
组建一个由不同专业背景的专家组成的团队，包括工艺工程师、安全工程师、操作人员、维护人员等。团队成员需要具备相关的知识和经验，以便从不同角度审视问题。

### 步骤3：确定分析节点
在流程图中选择关键的分析节点，这些节点通常是流程中的关键步骤或设备。在我们的氯气生产案例中，可能的节点包括氯气压缩机、冷却系统、储存罐等。

### 步骤4：应用引导词
针对每个节点，使用引导词与流程参数相结合，生成一系列的“偏离”场景。例如，对于氯气压缩机节点，我们可以使用引导词“没有”与参数“流量”结合，考虑“没有氯气流量”的情况。

### 步骤5：识别潜在问题
团队讨论每个偏离场景可能导致的后果，识别潜在的问题。例如，在“没有氯气流量”的情况下，可能会导致压缩机损坏或系统压力异常。

### 步骤6：提出建议
针对识别出的问题，团队提出相应的改进建议或预防措施。例如，对于“没有氯气流量”的问题，可能的建议包括安装流量监测器、设置报警系统、定期检查压缩机状态等。

### 步骤7：记录和审查
详细记录分析过程和结果，并进行审查，确保没有遗漏关键信息。这包括记录讨论的内容、提出的建议、采取的措施等。

### 步骤8：实施和跟踪
将提出的建议转化为实际的改进措施，并跟踪实施效果，确保风险得到有效控制。

### 步骤9：反向事故路径分析
使用反向事故路径分析，从事故结果逐级倒查，找到事故发生的真正原因。例如，如果发生了氯气泄漏事故，我们需要追溯到事故发生的根本原因，可能是设备老化、操作失误、维护不当等。

通过以上步骤，HAZOP分析能够系统性地识别潜在风险，并设计出有效的防范策略，以避免